<a href="https://colab.research.google.com/github/adilsonalbino/SQL-e-PYSPARK/blob/main/13_Operador_NOT_UNION_filtrando_dados_nulos_com_isNull_no_SQL_e_PYSPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#13-Operador-NOT-UNION-filtrando dados nulos com isNull no SQL e PYSPARK.ipynb

by Adilson Albino



In [1]:
#instalando pyspark
!pip install pyspark

#importando bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

#Criando app spark
spark = SparkSession.builder \
.appName("Spark")\
.getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=a7bc985e883eb0e781f0885b402edab029b8fbbaace9c90eabf8d099d05d4b90
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
#Variavel com a informação do caminho do dataset
file_path_carros = "/content/drive/MyDrive/Colab Notebooks/PySpark_SQL/DATASET_CARROS/modelo_carro.csv"

In [8]:
#Carregando o dataset
df_carros = spark\
.read.format("csv")\
.option("header", True)\
.load(file_path_carros)\

df_carros.show(5)

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
|       3|        Golf|$61274.55|       55|
|       4|          EX|$84981.12|       23|
|       5|      Escort|$77466.89|       17|
+--------+------------+---------+---------+
only showing top 5 rows



In [12]:
#Criando dois dataframes fitrados com where()

df_carros_1 = df_carros.where(
    (col("id_carro") == '1') |
    (col("id_Carro") == '2') |
    (col("id_carro") == '3')
)

df_carros_2 = df_carros.where(
    (col("id_carro") == '4')
).withColumn(
    "preco",
    lit(None)
)

#utilizando UNION
df_carros = df_carros_1.union(df_carros_2)
df_carros.show()

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
|       3|        Golf|$61274.55|       55|
|       4|          EX|     NULL|       23|
+--------+------------+---------+---------+



#Utilizando SQL

In [16]:
df_carros_sql = df_carros.createOrReplaceTempView("carros")

In [35]:
#UTILIZANDO (IS NULL)
df_carros_sql = spark.sql("""
SELECT
  *
FROM carros
WHERE preco IS NULL
""")
df_carros_sql.show()

+--------+------------+-----+---------+
|id_carro|modelo_carro|preco|cod_marca|
+--------+------------+-----+---------+
|       4|          EX| NULL|       23|
+--------+------------+-----+---------+



In [36]:
#UTILIZANDO (IS NOT NULL)
df_carros_sql = spark.sql("""
SELECT
  *
FROM carros
--WHERE id_carro NOT IN('1', '2', '3')
WHERE preco IS NOT NULL
""")
df_carros_sql.show()

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
|       3|        Golf|$61274.55|       55|
+--------+------------+---------+---------+



In [39]:
#UTILIZANDO (IN)
df_carros_sql = spark.sql("""
SELECT
  *
FROM carros
WHERE id_carro IN('1', '2')
""")
df_carros_sql.show()

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
+--------+------------+---------+---------+



In [40]:
#UTILIZANDO (NOT IN)
df_carros_sql = spark.sql("""
SELECT
  *
FROM carros
WHERE id_carro NOT IN('1', '2')
""")
df_carros_sql.show()

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       3|        Golf|$61274.55|       55|
|       4|          EX|     NULL|       23|
+--------+------------+---------+---------+



#Utilizando PYSPARK


In [43]:
#Utilizando IS NULL
df_carros_spark = df_carros.where(
    col("preco").isNull()
)
df_carros_spark.show()

+--------+------------+-----+---------+
|id_carro|modelo_carro|preco|cod_marca|
+--------+------------+-----+---------+
|       4|          EX| NULL|       23|
+--------+------------+-----+---------+



In [45]:
#Utilizando IS NOT NULL
df_carros_spark = df_carros.where(
    col("preco").isNotNull()
)
df_carros_spark.show()

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
|       3|        Golf|$61274.55|       55|
+--------+------------+---------+---------+



In [46]:
#Utilizando isin
df_carros_spark = df_carros.where(
    col("id_carro").isin(['1', '2'])
)
df_carros_spark.show()

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
+--------+------------+---------+---------+



In [51]:
#Utilizando ~ para negar a condição
df_carros_spark = df_carros.where(
    ~(col("id_carro").isin(['1', '2']))
)
df_carros_spark.show()

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       3|        Golf|$61274.55|       55|
|       4|          EX|     NULL|       23|
+--------+------------+---------+---------+

